# Paper Data Processing

## Mesh to SDF

In [ ]:
# The path to the mesh file
meshPath = r"/home/student/Documents/Data/Input_puntenwolken/synthetisch_input/vaten/vat_staand_6.obj"
saveFolder = r"/home/student/Documents/Data/meshes_data/synthetisch_meshes/vat"

voxelGridSize = 64

In [2]:
import trimesh
import os
import comptools
import pathlib
from alphashape import alphashape
mesh = trimesh.load(meshPath)
meshName = meshPath.split('\\')[-1][:-4]
meshName = os.path.basename(meshPath)[:-4]
savePath = os.path.join(saveFolder, meshName)
comptools.make_dir_if_not_exist(savePath)
print(savePath)

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.
/home/student/Documents/Data/meshes_data/synthetisch_meshes/vat/vat_staand_6


### Display the mesh (Optional)

In [4]:
import math
import trimesh.transformations as transformations

# Recenter the mesh to the origin
mesh.apply_translation(-mesh.bounding_box.centroid)

# Normalize the scale of the mesh
# Scale so the bounding box fits within a unit cube
bounding_box_extents = mesh.bounding_box.extents
scale_factor = 1.0 / max(bounding_box_extents)
mesh.apply_scale(scale_factor)

#angle = -math.pi/2
#direction = [1, 0, 0]
#direction = [1, 0, 1]
#center = [0, 0, 0]

#rot_matrix = transformations.rotation_matrix(angle, direction, center)

#mesh.apply_transform(rot_matrix)
mesh.show()

In [238]:
#Creating a alpha shape to create a more complex hull
#alphaValue = 2
#alphaMesh = alphashape(mesh.vertices, alphaValue)
#alphaMesh.invert()
#alphaMesh.show()

In [239]:
#mesh = alphaMesh

### SDF Creation

In [5]:

import torch
import numpy as np
import time

t0 = time.time() # keep track of how long the process takes
sdf,sdfMesh = comptools.mesh_to_sdf_tensor(comptools.as_mesh(mesh), voxelGridSize, recenter=True, scaledownFactor=8)
print(sdf.shape)
fixedMeshPath = os.path.join(savePath,"sdf.obj")
sdfPath = fixedMeshPath[:-4] + ".npy"
tfPath = fixedMeshPath[:-4] + ".pt"

# output
mesh.export(os.path.join(savePath, "gt.obj"))
sdfMesh.export(fixedMeshPath)
np.save(sdfPath, sdf)
# Tensor
shapedData = np.reshape(sdf, [1, 1, 64, 64, 64])
torch.save(torch.from_numpy(shapedData[:,:,:64,:64,:64]), tfPath)

t1 = time.time()
print('It takes %.4f seconds to process' % (t1-t0))


(64, 64, 64)
It takes 16.4912 seconds to process


In [6]:
from skimage.measure import marching_cubes
vertices, faces, normals, _ = marching_cubes(sdf, level=0)
sdfMesh = trimesh.Trimesh(vertices=vertices, faces=faces, vertex_normals=normals)
sdfMesh.show()